# Problem Set 3 (PS3): PhoR/PhoB Two-Component System
The PhoR/PhoB two-component system is a well-studied example of a TCS that regulates bacteria's response to phosphate limitation. PhoR is the sensor kinase that detects low phosphate levels, while PhoB is the response regulator that activates phosphate acquisition and metabolism genes. 

[PhoR/PhoB Cartoon, Figure 7.5 reproduced from the textbook, "Molecular Biology of the Cell" by Alberts et al. 6th Edition](https://github.com/varnerlab/CHEME-5450-Lectures-Spring-2025/blob/main/lectures/week-8/L8b/figs/figure%207-05.jpg).

Review of the PhoR/PhoB TCS in _Escherichia coli_:
* [Gardner SG, McCleary WR. Control of the phoBR Regulon in Escherichia coli. EcoSal Plus. 2019 Sep;8(2):10.1128/ecosalplus.ESP-0006-2019. doi: 10.1128/ecosalplus.ESP-0006-2019. PMID: 31520469; PMCID: PMC11573284.](https://pubmed.ncbi.nlm.nih.gov/31520469/)

In this problem set, let's build a simplified model of PhoR/PhoB mediated transcription of PhoA, a downtream target of the activated (phosphorylated) PhoB transcription factor. 

## Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 
* The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [1]:
include("Include.jl");

### Constants
See the description beside the constant for a description of what it is, permissible values, etc.

In [ ]:
doubling_time  = 20.0*(1/60); # doubling time of E. coli in hr
μ = log(2)/doubling_time; # maximum specific growth rate of E. coli in batch culture on LB media (units: 1/hr)
R = 8.314; # universal gas constant (units: J/(mol*K))
T = 37 + 273.15; # temperature (units: K);
β = 1/(R*T); # inverse temperature (units: 1/(J*mol)) - thermodynamic beta in units of 1/(J*mol)
parameters = generate_parameter_dictionary(joinpath(_PATH_TO_DATA, "Parameters.json")); # load the biophysical parameters

In [3]:
μ

2.079441541679836

### Build the problem object 
To store all the problem data, we created [the `MyPrimalFluxBalanceAnalysisCalculationModel` type](src/Types.jl). Let's build one of these objects for our problem and store it in the `model::MyPrimalFluxBalanceAnalysisCalculationModel` variable. We also return the `rd::Dict{String, String}` dictionary, which maps the reaction name field (key) to the reaction string (value).
* __Builder (or factory) pattern__: For all custom types that we make, we'll use something like [the builder software pattern](https://en.wikipedia.org/wiki/Builder_pattern) to construct and initialize these objects. The calling syntax will be the same for all types: [a `build(...)` method](src/Factory.jl) will take the kind of thing we want to build in the first argument, and the data needed to make that type as [a `NamedTuple` instance](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) in the second argument.
* __What's the story with the `let` block__? A [let block](https://docs.julialang.org/en/v1/manual/variables-and-scoping/#Let-Blocks) creates a new hard scope and new variable bindings each time they run. Thus, they act like a private scratch space, where data comes in (is captured by the block), but only what we want to be exposed comes out. 

In [4]:
model, rd, reactionnamesmap = let

    # first, load the reaction file - and process it
    listofreactions = read_reaction_file(joinpath(_PATH_TO_DATA, "PS3-PHOB-CHEME-5450-S2025.net")); # load the reactions from the VFF reaction file
    S, species, reactions, rd = build_stoichiometric_matrix(listofreactions); # Builds the stochiometric matrix, species list, and the reactions list
    boundsarray = build_default_bounds_array(listofreactions); # Builds a default bounds model using the flat file flags

    # build the FBA model -
    model = build(MyPrimalFluxBalanceAnalysisCalculationModel, (
        S = S, # stoichiometric matrix
        fluxbounds = boundsarray, # these are the *default* bounds; we'll need to update with new info if we have it
        species = species, # list of species. The rows of S are in this order
        reactions = reactions, # list of reactions. The cols of S are in this order
        objective = length(reactions) |> R -> zeros(R), # this is empty, we'll need to set this
    ));

    # build a reaction names = reaction index map
    reactionnamesmap = Dict{String,Int64}()
    for i ∈ eachindex(reactions)
        reactionnamesmap[reactions[i]] = i;
    end

    # return -
    model, rd, reactionnamesmap
end;

`Unhide` the code block below to see how we build a table of the reactions in the model [using the `pretty_tables(...)` method exported from the `PrettyTables.jl` package](https://github.com/ronisbr/PrettyTables.jl).

In [5]:
let
    df = DataFrame()
    reactions = model.reactions;

    for i ∈ eachindex(reactions)
        reactionstring = reactions[i] |> key -> rd[key];
        row_df = (
            name = reactions[i],
            string = reactionstring,
        );
        push!(df, row_df);
    end

    pretty_table(df, tf = tf_simple, alignment = :l)
end

============================ ===========================================================
  name                        string                                                   
  String                      String                                                   
============================ ===========================================================
  PhoR_SYNTHESIS              [] = PhoR
  PhoR_MEMBRANE               PhoR = PhoR_MEMBRANE
  PhoR_DEGRADATION            PhoR = []
  PhoR_MEMBRANE_DEGRADATION   PhoR_MEMBRANE = []
  PhoB_SYNTHESIS              [] = PhoB
  PhoB_DEGRADATION            PhoB = []
  RIBOSOME_ASSEMBLY           [] = RIBOSOME
  RIBOSOME_FORCE_RECYCLE      RIBOSOME_USED = []
  RNAP_ASSEMBLY               [] = RNAP
  RNAP_FORCE_RECYCLE          RNAP_USED = []
  ATP_SYNTHESIS               [] = ATP
  ADP_SYNTHESIS               [] = ADP
  BIND_PhoR_ATP               PhoR_MEMBRANE+ATP = PhoR_MEMBRANE_ATP
  ACTIVATE_PhoB_TRANSFER      PhoR_MEMBRANE_ATP_PhoB = PhoR_ME

Update the objective function. Select a process to maximize. Let's maximize the translation of the PhoA protein.

In [6]:
i = findfirst(x-> x=="PHOA_TRANSLATION", model.reactions);
objective = model.objective;
objective[i] = 1;

## The bounds trick for signaling
The signaling events in the PhoR/PhoB TCS are modeled in our network, but we need to add some logic to the bounds to make the model work correcrly. The initiation event for the activation of PhoB is the (auto) phosphorylation of the PhoR sensor kinase, which occurs when the sensor kinase does _not_ have a bound perplasmic phosphate. Let's model this use boolean rules.

### The `PhoR` sensor kinase
The `PhoR` sensor kinase is phosphorylated (activated) when the concentration of periplasmic phosphate is below a threshold. Let this condition by modeled by the binary variable $\texttt{HIGHPHOSPHATE}$. 
* If $\texttt{HIGHPHOSPHATE} = 1$: the sensor kinase will _not_ auto-phosphorylate. The periplasmic phosphate abundance is high, so the sensor kinase is _not_ activated.
* If $\texttt{HIGHPHOSPHATE} = 0$: the sensor kinase _will_ auto-phosphorylate, so the sensor kinase is _activated_ in low phosphate conditions.

### The `PhoB` response regulator
The `PhoB` response regulator is phosphorylated (activated) when PhoR is phosphorylated. Let this condition be modeled by the fuzzy variable $\texttt{PHOSPHORYLATED\_PHOB}$. We expect that PhoB will be phosphorylated in low phosphate conditions.
* Let $\texttt{PHOSPHORYLATED\_PHOB} = 1 - \texttt{HIGHPHOSPHATE}$. In high phosphate conditions, the response regulator is _not_ phosphorylated, and the `BIND_PhoB_PhoR` reaction is _inactive_, thus, $\texttt{PHOSPHORYLATED\_PHOB} = 0$. Otherwise, $\texttt{PHOSPHORYLATED\_PHOB} = 1$.

Let's implement this logic, and we'll update the bounds below.

In [14]:
HIGHPHOSPHATE = 1; # we are in a high phoshate environment
PHOSPHORYLATED_PHOB = 1 - HIGHPHOSPHATE; # we are in a low phosphate environment 

## The bounds trick for gene expression in FBA

In general, when using flux balance analysis to model gene expression there can be some pathologies. For example, translation occurs without transcription. Alternatively, transcription occurs without PhoB activation. How are we going to fix this? Answer: there is a trick with the bounds (that incorporates many things we have been exploring) that we can use to fix this problem:
* [Vilkhovoy M, Horvath N, Shih CH, Wayman JA, Calhoun K, Swartz J, Varner JD. Sequence-Specific Modeling of E. coli Cell-Free Protein Synthesis. ACS Synth Biol. 2018 Aug 17;7(8):1844-1857. doi: 10.1021/acssynbio.7b00465. Epub 2018 Jul 16. PMID: 29944340.](https://www.biorxiv.org/content/10.1101/139774v2)

__Fix__: We (equality) bound a transcription rate $\hat{v}_{i}$ as $\hat{r}_{X, i}u_{i} = \hat{v}_{i} = \hat{r}_{X, i}u_{i}$, while a translation rate is bounded from above by the modified kinetic limit as: $0\leq\hat{v}_{j}\leq\hat{r}_{L,j}w_{j}$. This is interesting because the kinetic limits are (semi)mechanistic descriptions of the transcription and translation rate. At the same time, the control variables contain continuous Boltzmann-type descriptions of the logical controlling these processes.

First, let's compute the kinetic limit of transcription for PhoA:

In [7]:
rX = compute_transcription_rate(parameters); # transcription kinetic limit, no u

Next, we'll compute the transcriptional and translational control terms that govern PhoA expression. For simplicity, let's assume the translation control term is unity $w = 1$, and focus on a Boltzmann-type description of the transcriptional control term $u$. 

Let's assume three states:
* __State 0: Bare gene__: This state is just the gene without anything bound to it. This state will __not__ lead to transcription. This will be our ground state. The pseduo energy for this state $\epsilon_{0} \equiv 0$.
* __State 1: RNAP only__: In this state only RNAP is bound to the promoter, but without phosphorylated PhoB (P_PhoB) bound. This state __will__ lead to transcription at a low level. The pseduo energy for this state $\epsilon_{1} \equiv x$.
* __State 2: RNAP + P_PhoB__: In this state both RNAP and P_PhoB are bound to the promoter. This state __will__ lead to transcrption that will be sensitive to the abundance of P_PhoB. The pseduo energy for this state $\epsilon_{2} \equiv y$.

In [ ]:
u₁, u₂, w = let 
    
    # initialize -
    w = 1.0;
    u₁ = 0.0; # this is udagger (background)
    u₂ = 0.0; # this is u (induced)

    # Boltzmann promoter logic here -
    # ...

    # return -
    u₁,u₂,w
end

(1, 1, 1)

Finally, let's compute the kinetic limit of translation. This one is tricky because it requires an estimate of the mRNA for PhoA.
See lecture `L5b` for a description of the kinetic limit of translation expression (or the reference we gave above). Let's approximate the PhoA mRNA level by the steady-state level (written for transcript $j$):
$$
\begin{align*}
 m^{\star}_{j} & = \frac{r_{X,j}u_{j}\left(\dots\right) + \lambda_{j}}{\theta_{m,j}+\mu}\quad\text{for }j=1,2,\dots,N
\end{align*}
$$

In [9]:
rL = let

    # get constants from parameters -
    θ = parameters[:kdL];
    KL = parameters[:KL];
    VMAXL = parameters[:VL];
    τ = parameters[:L_tau_factor]; 

    # first compute the mRNA concentration -
    m = (rX*u₁ + rX*u₂)/(θ + μ); # approx mRNA level -

    # compute the kinetic limit -
    rL = VMAXL*(m/(τ*KL+(1+τ)*m));
end

0.2649521199960554

### Update the bounds
Now that we have an estimate of the transcription $\hat{r}_{X}u$ and the translation $\hat{r}_{L}w$ rates, we can update the flux balance analysis problem bounds. Let's play around with these bounds to see what happens.

In [10]:
fluxbounds = let

    # make a copy of the default flux bounds -
    flux_bounds = copy(model.fluxbounds);

    # update the bounds -
    flux_bounds[reactionnamesmap["PHOA_START"],1] = rX*u₁; # transcrption lower bound state 1
    flux_bounds[reactionnamesmap["PHOA_START"],2] = rX*u₁; # transcrption upper bound state 1
    flux_bounds[reactionnamesmap["PHOA_TF_START"],1] = rX*u₂; # transcrption lower bound state 2
    flux_bounds[reactionnamesmap["PHOA_TF_START"],2] = rX*u₂; # transcrption upper bound state 2
    flux_bounds[reactionnamesmap["PHOA_TRANSLATION"],1] = 0.0; # translation lower bound
    flux_bounds[reactionnamesmap["PHOA_TRANSLATION"],2] = rL*w; # translation upper bound

    # return new bounds
    flux_bounds;
end;

In [11]:
model.fluxbounds = fluxbounds;

### Compute the optimal flux distribution 

Finally, let's compute the optimal metabolic distribution $\left\{\hat{v}_{i} \mid i = 1,2,\dots,\mathcal{R}\right\}$ by solving the [linear programming problem](). We solve the optimization problem by passing the `model::MyPrimalFluxBalanceAnalysisCalculationModel` to [the `solve(...)` method](src/Compute.jl). This method returns a `solution::Dict{String, Any}` dictionary, which holds information about the solution.
* __Why the [try-catch environment](https://docs.julialang.org/en/v1/base/base/#try)__? The [solve(...) method](src/Compute.jl) has an [@assert statement](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check if the calculation has converged. Thus, the solve method can [throw](https://docs.julialang.org/en/v1/base/base/#Core.throw) an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) if the optimization problem fails to converge. To gracefully handle this case, we use a [try-catch construct](https://docs.julialang.org/en/v1/base/base/#try). See the [is_solved_and_feasible method from the JuMP package](https://jump.dev/JuMP.jl/stable/api/JuMP/#JuMP.is_solved_and_feasible) for more information.

In [12]:
solution = let
    
    solution = nothing; # initialize nothing for the solution
    try
        solution = solve(model); # call the solve method with our problem model -
    catch error
        println("error: $(error)"); # Oooooops! Looks like we have a *major malfunction*, problem didn't solve
    end

    # return solution
    solution
end;

__Flux table__: Let's use [the `pretty_tables(...)` method exported by the `PrettyTables.jl` package](https://github.com/ronisbr/PrettyTables.jl) to display the estimated optimal metabolic fluxes. `Unhide` the code block below to see how we constructed the flux table.

In [13]:
let

    # setup -
    S = model.S;
    flux_bounds_array = model.fluxbounds;
    number_of_reactions = size(S,2); # columns
	flux_table = Array{Any,2}(undef,number_of_reactions,5)
    flux = solution["argmax"];
    
    # populate the state table -
	for reaction_index = 1:number_of_reactions
		flux_table[reaction_index,1] = model.reactions[reaction_index]
		flux_table[reaction_index,2] = flux[reaction_index]
		flux_table[reaction_index,3] = flux_bounds_array[reaction_index,1]
		flux_table[reaction_index,4] = flux_bounds_array[reaction_index,2]
        flux_table[reaction_index,5] = model.reactions[reaction_index] |> key-> rd[key]
	end

    # header row -
	flux_table_header_row = (["Reaction","v̂ᵢ", "v̂ᵢ LB", "v̂ᵢ UB", "Reaction"],["","mmol/gDW-time", "mmol/gDW-time", "mmol/gDW-time", "N/A"]);
		
	# write the table -
	pretty_table(flux_table; header=flux_table_header_row, tf=tf_simple, alignment = :l)
end

============================ =============== =============== =============== ===========================================================
  Reaction                    v̂ᵢ              v̂ᵢ LB           v̂ᵢ UB           Reaction                                                 
                              mmol/gDW-time   mmol/gDW-time   mmol/gDW-time   N/A                                                      
============================ =============== =============== =============== ===========================================================
  PhoR_SYNTHESIS              0.0             0.0             1000.0          [] = PhoR
  PhoR_MEMBRANE               0.0             -1000.0         1000.0          PhoR = PhoR_MEMBRANE
  PhoR_DEGRADATION            0.0             0.0             1000.0          PhoR = []
  PhoR_MEMBRANE_DEGRADATION   0.0             0.0             1000.0          PhoR_MEMBRANE = []
  PhoB_SYNTHESIS              0.0             0.0             1000.0          [